# IS_AGGRESIEVE_POSE with OPENPOSE

### Concept Overview 
1. Pose Estimation: 
    * Pose estimation invovles detecting key points (joints) on the human body in an image, such as the head, shoulders, elbows and knees 
    * Tools like OpenpOse ouput these key points along with their coordinates 
2. Defining "Aggressive" Poses: 
    * Raised hands holding a weapon 
    * Swining arms (indicative of hitting or threatening behavior)
    * a stance suggesting an attacj (leaning forward, legs wide apart )
3. Analyzing Pose Data: 
    * key points can be analyzed to infer these beahviors. For example:
        - Raised hands: The y-coordinate of hand joints is higher than the shoulder joints
        - Leaning forward: the neck joints is fat ahead of the jip joint on the x-axis

### Steps to Implement 
1. Understand the pose Data Format 